# Import

In [3]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
OPENAI = os.getenv("OPENAI_KEY")
from openai import OpenAI
from tqdm import tqdm
import json
import re


### Cleaning

In [244]:
# Clean the unfiltered data
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df_clean = df[df["reviews"] != "none"]
    df_clean.drop(["business_status", "icon", "icon_background_color", "icon_mask_base_uri", "opening_hours"], inplace=True, axis=1)    
    
    for elem in df["reviews"]:
        for review in elem:
            try:
                del review["link"]
                del review["source"]
                del review["review_id"]
                del review["user"]
                del review["images"]
            except:
                print("Key not found")
                pass

    return df_clean

In [245]:
df = pd.read_json("data-with-reviews.json")
df = clean_df(df)

Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key no

/var/folders/1t/0999bgxs5s17vns3pvsxhgr80000gn/T/ipykernel_82224/2873737295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(["business_status", "icon", "icon_background_color", "icon_mask_base_uri", "opening_hours"], inplace=True, axis=1)


In [253]:
client = OpenAI(api_key=OPENAI)

def get_embedding(text: str, model="text-embedding-ada-002") -> np.ndarray:
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def compute_all_embeddings(df_: pd.DataFrame, engine_: str) -> pd.DataFrame:
    for idx, text in enumerate(df_['reviews']):
        try:
            df_.at[idx, 'embeddings'] = get_embedding(str(text), engine_)
        except Exception as e:
            print(f'Error: {e}')
            df_.at[idx, 'embeddings'] = "~"
    return df_

In [254]:
engine="text-embedding-ada-002"
df["embeddings"] = "no_embeddings"
train_df = compute_all_embeddings(df, engine)
# train_df

Error: Must have equal len keys and value when setting with an iterable
Error: Must have equal len keys and value when setting with an iterable
Error: Must have equal len keys and value when setting with an iterable
Error: Must have equal len keys and value when setting with an iterable
Error: Must have equal len keys and value when setting with an iterable


In [14]:
df = pd.read_csv("data-with-embeddings.csv")

In [15]:
# # extract zip code
df['postal_code'] = df['formatted_address'].apply(lambda x: re.findall(r'\b\d{5}\b', x)[0] if re.findall(r'\b\d{5}\b', x) else None)
df.drop("formatted_address", inplace=True, axis=1)
# drop photos (all unique and non null)
df.drop(["photos"], inplace=True, axis=1)
# topics to boolean
df.loc[df["topics"] == "no topics", "topics"] = False
df.loc[df["topics"] != False, "topics"] = True
# extract lat and long
df['lat'] = df['geometry'].str.extract(r"'lat': (\d+\.\d+)")
df['lng'] = df['geometry'].str.extract(r"'lng': (\d+\.\d+)")
df.drop("geometry", inplace=True, axis=1)
# drop other cols
df.drop(["place_id", "plus_code", "reference"], inplace=True, axis=1)

In [16]:
df.head(-1)

,name,rating,types,user_ratings_total,reviews,topics,embeddings,postal_code,lat,lng
0,Dental Centre Turkey,4.7,"['dentist', 'health', 'point_of_interest', 'es...",253.0,"[{'rating': 4.0, 'date': '2 months ago', 'snip...",True,"[-0.014175066724419594, 0.007809185888618231, ...",07230,36.8617744,30.7920385
1,Queen Dental,4.8,"['dentist', 'health', 'point_of_interest', 'es...",135.0,"[{'rating': 5.0, 'date': '6 days ago', 'snippe...",True,"[-0.014537202194333076, 0.009266605600714684, ...",07600,36.8939721,30.6910022
2,Dental Clinic Antalya,4.8,"['dentist', 'doctor', 'health', 'point_of_inte...",90.0,"[{'rating': 5.0, 'date': '10 months ago', 'sni...",True,"[-0.028139766305685043, 0.0001762386818882078,...",07050,36.886999,30.683048
3,"DENTAL DESIGN TURKEY (Dentist Antalya, Dental ...",4.9,"['dentist', 'health', 'point_of_interest', 'es...",216.0,"[{'rating': 5.0, 'date': '2 weeks ago', 'snipp...",True,"[-0.02719695121049881, 0.006841208320111036, 0...",07160,36.8526995,30.7517603
4,Dental Centre Turkey,4.7,"['dentist', 'health', 'point_of_interest', 'es...",253.0,"[{'rating': 4.0, 'date': '2 months ago', 'snip...",True,"[-0.014175066724419594, 0.007809185888618231, ...",07230,36.8617744,30.7920385
...,...,...,...,...,...,...,...,...,...,...
775,Medic Antalya Dental Clinic,5.0,"['health', 'point_of_interest', 'establishment']",100.0,"[{'rating': 5.0, 'date': '6 months ago', 'snip...",False,"[0.001659317291341722, -0.025826003402471542, ...",07160,36.8575347,30.7579335
776,Kotul Healthcare,5.0,"['dentist', 'health', 'point_of_interest', 'es...",19.0,"[{'rating': 5.0, 'date': '3 months ago', 'snip...",False,"[0.001659317291341722, -0.025826003402471542, ...",07200,36.8811601,30.7248129
777,Premium Dental Turkey,4.9,"['dentist', 'doctor', 'health', 'point_of_inte...",231.0,"[{'rating': 5.0, 'date': '2 weeks ago', 'snipp...",False,"[0.001659317291341722, -0.025826003402471542, ...",07200,36.8638292,30.7432691
778,Dental Support Turkey,5.0,"['health', 'point_of_interest', 'establishment']",30.0,"[{'rating': 5.0, 'date': '4 months ago', 'snip...",False,"[0.001659317291341722, -0.025826003402471542, ...",07700,36.8612492,30.7822271


# Findings
- \> 50 reviews looks to be negative
- "point_of_interest" does not matter